In [4]:
# Parámetro para generar el modelo del taggeador
data = False
%store data

Stored 'data' (bool)


In [5]:
%%time
%run ./carga_diputados.ipynb

Wall time: 4.19 s


In [6]:
%%time
%run ./carga_dialogos.ipynb

False


Documentos.....:   0%|          | 0/11 [00:00<?, ?it/s]

PERSONA NO ENCONTRADA: 
sánchez pérez castejón
00_INVESTIDURA
PERSONA NO ENCONTRADA: 
sánchez pérez castejón
00_INVESTIDURA


KeyboardInterrupt: 

Wall time: 4min 37s
